In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import selenium
import itertools
import pandas as pd
import numpy as np
import time

In [2]:
link="https://fingertips.phe.org.uk/profile/general-practice/data#page/0/gid/2000006/pat/167/par/E38000220/ati/7/iid/253/age/1/sex/4/cat/-1/ctp/-1/yrr/1/cid/4/tbm/1"

In [3]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 103.0.5060
Get LATEST chromedriver version for 103.0.5060 google-chrome
Driver [/Users/jovansandhu/.wdm/drivers/chromedriver/mac64/103.0.5060.53/chromedriver] found in cache


In [4]:
browser.visit(link)

In [5]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [6]:
button = browser.find_by_xpath('/html/body/ft-data-view/div[1]/div/div[2]/ft-geography-summary/div/div[2]/button/div[1]/div[2]').click()

In [7]:
button = browser.find_by_xpath('/html/body/ngb-popover-window/div[2]/ft-popover-area-options/div/button[3]/div/div/div[1]').click()

In [8]:
gp_list_original = browser.html
gp_html = BeautifulSoup(gp_list_original, 'html.parser')
gp_labels=gp_html.body.find_all('button',class_="col-12 popover-button")

In [9]:
gp=[]
for i in gp_labels:
    gp.append(i.text)

In [10]:
gp

['NHS Barnsley CCG',
 'NHS Basildon And Brentwood CCG',
 'NHS Bassetlaw CCG',
 'NHS Bath and North East Somerset, Swindon and Wiltshire CCG',
 'NHS Bedfordshire, Luton and Milton Keynes CCG',
 'NHS Berkshire West CCG',
 'NHS Birmingham and Solihull CCG',
 'NHS Black Country and West Birmingham CCG',
 'NHS Blackburn With Darwen CCG',
 'NHS Blackpool CCG',
 'NHS Bolton CCG',
 'NHS Bradford District and Craven CCG',
 'NHS Brighton And Hove CCG',
 'NHS Bristol, North Somerset and South Gloucestershire CCG',
 'NHS Buckinghamshire CCG',
 'NHS Bury CCG',
 'NHS Calderdale CCG',
 'NHS Cambridgeshire and Peterborough CCG',
 'NHS Cannock Chase CCG',
 'NHS Castle Point And Rochford CCG',
 'NHS Cheshire CCG',
 'NHS Chorley And South Ribble CCG',
 'NHS County Durham CCG',
 'NHS Coventry and Warwickshire CCG',
 'NHS Derby and Derbyshire CCG',
 'NHS Devon CCG',
 'NHS Doncaster CCG',
 'NHS Dorset CCG',
 'NHS East And North Hertfordshire CCG',
 'NHS East Lancashire CCG',
 'NHS East Leicestershire And Ru

In [12]:
gp_percentage=[]
for i in range(len(gp)):
    
    print(i)
    button = browser.find_by_xpath('/html/body/ft-data-view/div[1]/div/div[2]/ft-geography-summary/div/div[2]/button/div[1]/div[2]').click()

    time.sleep(2)
    button = browser.find_by_xpath('/html/body/ngb-popover-window/div[2]/ft-popover-area-options/div/button[3]/div/div/div[1]').click()

    time.sleep(2)
    path=f"/html/body/ngb-popover-window/div[2]/ft-popover-area-options/ft-area-options/div/div/div[{i+1}]/button"
    browser.find_by_xpath(path).click()

    time.sleep(2)
    html = browser.html
    page = BeautifulSoup(html, 'html.parser')
    data=page.body.find_all('div', class_="tartan-text")[19].text.split("%")[0]
    gp_percentage.append(data)
    
    
    
browser.quit()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105


In [13]:
import requests
from config import GOOGLE
api_key=GOOGLE
gp_lat=[]
gp_lng=[]
url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"
for gps in gp:
    params={"key":api_key,
            'query':gps,
            'type':'health'
            }
    temp = requests.get(url, params=params).json()
    lat=temp['results'][0]['geometry']['location']['lat']
    lng=temp['results'][0]['geometry']['location']['lng']
    gp_lat.append(lat)
    gp_lng.append(lng)

In [14]:
df=pd.DataFrame(zip(gp,gp_lat,gp_lng,gp_percentage),columns=["gp_practice","latitude","longitude","asthma_percentage"])


In [15]:
from config import OWM
Weather_API=OWM
aqi=[]
co=[]
no=[]
no2=[]
o3=[]
so2=[]
pm2_5=[]
pm10=[]
nh3=[]

weather_url="http://api.openweathermap.org/data/2.5/air_pollution/history?"

for row in df.iterrows():
    
    params={"appid":Weather_API,
            'lat':row[1][1],
            'lon':row[1][2],
            'start':"1606266000",
            'end':"1637798400"
            }
    
    temp = requests.get(weather_url, params=params).json()
    
    avg1=0
    avg2=0
    avg3=0
    avg4=0
    avg5=0
    avg6=0
    avg7=0
    avg8=0
    avg9=0
    count=0
    for i in temp['list']:
        count+=1
        avg1=avg1+int(i['main']['aqi'])
        avg2=avg2+int(i['components']['co'])
        avg3=avg3+int(i['components']['no'])
        avg4=avg4+int(i['components']['no2'])
        avg5=avg5+int(i['components']['o3'])
        avg6=avg6+int(i['components']['so2'])
        avg7=avg7+int(i['components']['pm2_5'])
        avg8=avg8+int(i['components']['pm10'])
        avg9=avg9+int(i['components']['nh3'])
    
    aqi.append(round(avg1/count,2))
    co.append(round(avg2/count,2))
    no.append(round(avg3/count,2))
    no2.append(round(avg4/count,2))
    o3.append(round(avg5/count,2))
    so2.append(round(avg6/count,2))
    pm2_5.append(round(avg7/count,2))
    pm10.append(round(avg8/count,2))
    nh3.append(round(avg9/count,2))

In [16]:
df['aqi']=aqi
df['co']=co
df['no']=no
df['no2']=no2
df['o3']=o3
df['so2']=so2
df['pm2_5']=pm2_5
df['pm10']=pm10
df['nh3']=nh3

In [17]:
df

,gp_practice,latitude,longitude,asthma_percentage,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3
0,NHS Barnsley CCG,53.558921,-1.496626,6.6,1.27,231.93,1.15,11.38,52.85,2.58,3.59,4.81,1.25
1,NHS Basildon And Brentwood CCG,51.591158,0.495369,6.2,1.42,239.92,2.26,13.97,51.42,4.38,5.77,7.43,0.52
2,NHS Bassetlaw CCG,53.326042,-0.954687,6.4,1.29,233.11,0.64,10.13,54.02,1.81,3.74,5.17,1.97
3,"NHS Bath and North East Somerset, Swindon and ...",51.357787,-2.372918,6.9,1.34,226.67,0.39,7.60,56.45,1.15,4.53,5.85,1.69
4,"NHS Bedfordshire, Luton and Milton Keynes CCG",51.879624,-0.414266,6.2,1.34,233.05,1.29,11.76,51.54,2.58,4.95,6.45,0.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,NHS West Leicestershire CCG,52.769765,-1.205220,6.8,1.30,230.42,0.75,9.74,54.16,2.04,4.08,5.45,1.68
102,NHS West Suffolk CCG,52.251444,0.696794,7.4,1.37,228.69,0.20,6.89,56.91,1.08,5.05,6.66,1.68
103,NHS West Sussex CCG,50.815480,-0.367546,6.9,1.49,232.34,0.67,11.29,62.17,3.45,6.27,8.16,0.55
104,NHS Wigan Borough CCG,53.544953,-2.629185,7.6,1.28,232.62,2.24,14.06,49.63,3.20,3.88,5.00,1.40


In [18]:
GP_dict=[]
for row in df.iterrows():
    temp_dict={"Practice Name":row[1][0],
    "Latitude":row[1][1],
    "Longitude":row[1][2],
    "Asthma Percentage":row[1][3],
    "aqi":row[1][4],
    "co":row[1][5],
    "no":row[1][6],
    "no2":row[1][7],
    "o3":row[1][8],
    "so2":row[1][9],
    "pm2_5":row[1][10],
    "pm10":row[1][11],
    "nh3":row[1][12]
              }
    GP_dict.append(temp_dict)

In [19]:
GP_dict

[{'Practice Name': 'NHS Barnsley CCG',
  'Latitude': 53.558921,
  'Longitude': -1.4966262,
  'Asthma Percentage': '6.6',
  'aqi': 1.27,
  'co': 231.93,
  'no': 1.15,
  'no2': 11.38,
  'o3': 52.85,
  'so2': 2.58,
  'pm2_5': 3.59,
  'pm10': 4.81,
  'nh3': 1.25},
 {'Practice Name': 'NHS Basildon And Brentwood CCG',
  'Latitude': 51.59115809999999,
  'Longitude': 0.4953693,
  'Asthma Percentage': '6.2',
  'aqi': 1.42,
  'co': 239.92,
  'no': 2.26,
  'no2': 13.97,
  'o3': 51.42,
  'so2': 4.38,
  'pm2_5': 5.77,
  'pm10': 7.43,
  'nh3': 0.52},
 {'Practice Name': 'NHS Bassetlaw CCG',
  'Latitude': 53.3260422,
  'Longitude': -0.9546871000000001,
  'Asthma Percentage': '6.4',
  'aqi': 1.29,
  'co': 233.11,
  'no': 0.64,
  'no2': 10.13,
  'o3': 54.02,
  'so2': 1.81,
  'pm2_5': 3.74,
  'pm10': 5.17,
  'nh3': 1.97},
 {'Practice Name': 'NHS Bath and North East Somerset, Swindon and Wiltshire CCG',
  'Latitude': 51.35778699999999,
  'Longitude': -2.372918,
  'Asthma Percentage': '6.9',
  'aqi': 1.34,

In [20]:
df.to_csv("national_gp.csv",index=False)

In [ ]:
round(avg1/count,2)